<a href="https://colab.research.google.com/github/therab6it/UCSD_CSE151A-SS1/blob/Milestone-2/Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Submission for Milestone 2.

> Commit changes to the GitHub repo by clicking on *File > Save a copy in GitHub*. Select the correct repository (**/UCSD_151A-SS1**) and branch (**Milestone-2**) and include a meaningful commit message.



In [1]:
print("ö")

ö
